In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
movie_df = pickle.load(open('clean_movie_df.pickle', 'rb'))

In [3]:
#read holidays csv
holidays = pd.read_csv('usholidays.csv')

#convert all dates to datetime/timestamp
movie_df.release = pd.to_datetime(movie_df.release)
holidays.loc[:,'Date'] = pd.to_datetime(holidays.Date)

#filter out holiday dates before the year 2000
holidays = holidays[holidays.Date >= dt.datetime(2000,1,1)].Date

def find_closest_holiday(date):
    """
    Finds the days to the nearest US Federal Holiday
    args:
    date (timestamp)
    returns:
    nearest_hol (int) days to the nearest holiday
    """
    nearest_hol = 365
    for hol in holidays:
        d = abs((hol - date).days)
        if d < nearest_hol:
            nearest_hol = d
    return nearest_hol

movie_df['holiday_distance'] = movie_df.release.map(find_closest_holiday)

In [4]:
movie_df.describe()

,holiday_distance
count,4195.000000
mean,12.548510
std,11.321007
min,0.000000
25%,4.000000
50%,10.000000
75%,17.000000
max,52.000000


In [14]:
#Get dummies from Genres
movie_df = movie_df.merge(movie_df.genres.str.get_dummies(sep=','), left_index=True, right_index=True, );

In [15]:
#Get dummies from movie distributor
movie_df = movie_df.merge(pd.get_dummies(movie_df.distr), left_index=True, right_index=True);

In [17]:
movie_df.drop(columns=['link_stub','desc','genres'], inplace=True)

In [18]:
with open('processed_movie_df.pickle', 'wb') as to_write:
    pickle.dump(movie_df, to_write)